In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler
from imblearn.ensemble import BalancedRandomForestClassifier

In [3]:
columns = [
    'movie_id', 'title', 'release_date', 'movie_popularity', 
    'vote_average', 'vote_count', 'budget', 'revenue', 'ratings', 'genres',
    'studios', 'actor_popularity_mean', 'director_popularity_mean'
]

target = ['revenue']

In [4]:
# Load the data
file_path = Path('../Tables/meta_ml.csv')
df = pd.read_csv(file_path)[:-2]
# , index_col=0
# df = df.loc[:, columns].copy()

# # Drop the null columns where all values are null
# df = df.dropna(axis='columns', how='all')

# # Drop the null rows
# df = df.dropna()

# Encoded Age Ratings
ratings_enc = {
    'G': 1,
    'PG': 2,
    'PG-13': 3,
    'R': 4,
    'NC-17': 5,
    'NR':6
}

df['ratings_enc'] = df['ratings'].apply(lambda x: ratings_enc[x])

df['release_month'] = pd.DatetimeIndex(df['release_date']).month
df.head()

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,studios,genres,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.993965,2.566,2,11
1,557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,"[''Other'', '' Columbia Pictures'', '' Sony Pi...","[''Fantasy'', ''Action'']",2.387022,2.914,3,5
2,672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.696712,2.566,2,11
3,673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,"[''Warner Bros. Pictures'', '' Other'', '' Hey...","[''Adventure'', ''Fantasy'']",3.523069,3.333,2,5
4,674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'', ''Family'']",3.234944,2.695,3,11


In [5]:
def profitability_df(df):
    if (df['revenue'] >= df['budget']*2):
        return True
    else:
        return False

df['profitability'] = df.apply(profitability_df, axis = 1)
df

,movie_id,title,release_date,movie_popularity,vote_average,vote_count,budget,revenue,ratings,studios,genres,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,profitability
0,671,Harry Potter and the Philosopher's Stone,2001-11-16,268.472,7.9,21429,125000000,976475550,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.993965,2.566,2,11,True
1,557,Spider-Man,2002-05-01,480.954,7.2,14421,139000000,821708551,PG-13,"[''Other'', '' Columbia Pictures'', '' Sony Pi...","[''Fantasy'', ''Action'']",2.387022,2.914,3,5,True
2,672,Harry Potter and the Chamber of Secrets,2002-11-13,246.027,7.7,17294,100000000,876688482,PG,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'']",2.696712,2.566,2,11,True
3,673,Harry Potter and the Prisoner of Azkaban,2004-05-31,225.882,8.0,17001,130000000,789804554,PG,"[''Warner Bros. Pictures'', '' Other'', '' Hey...","[''Adventure'', ''Fantasy'']",3.523069,3.333,2,5,True
4,674,Harry Potter and the Goblet of Fire,2005-11-16,244.428,7.8,16341,150000000,895921036,PG-13,"[''Warner Bros. Pictures'', '' Heyday Films'',...","[''Adventure'', ''Fantasy'', ''Family'']",3.234944,2.695,3,11,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,370172,No Time to Die,2021-09-29,3366.389,7.6,2075,242000000,734000000,PG-13,"[''Other'', '' Metro-Goldwyn-Mayer'', '' Other...","[''Adventure'', ''Action'', ''Thriller'']",4.164382,4.167,3,9,True
68,580489,Venom: Let There Be Carnage,2021-09-30,5797.863,7.0,2452,110000000,454000000,PG-13,"[''Marvel Entertainment'', '' Pascal Pictures'...","[''Science Fiction'', ''Action'', ''Adventure'']",4.363038,13.077,3,9,True
69,610253,Halloween Kills,2021-10-14,616.978,6.9,1251,20000000,127000000,R,"[''Universal Pictures'', '' Other'', '' Other'...","[''Horror'', ''Thriller'']",3.464000,2.385,4,10,True
70,576845,Last Night in Soho,2021-10-21,685.843,7.5,458,43000000,19000000,R,"[''Other'', '' Other'', '' Other'', '' Other''...","[''Horror'', ''Mystery'', ''Thriller'']",2.869283,7.468,4,10,False


In [6]:
df = df.drop(['release_date', 'title', 'ratings', 'revenue'], axis=1)
df = df.drop(['studios', 'genres'], axis=1)
df.head()

,movie_id,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month,profitability
0,671,268.472,7.9,21429,125000000,2.993965,2.566,2,11,True
1,557,480.954,7.2,14421,139000000,2.387022,2.914,3,5,True
2,672,246.027,7.7,17294,100000000,2.696712,2.566,2,11,True
3,673,225.882,8.0,17001,130000000,3.523069,3.333,2,5,True
4,674,244.428,7.8,16341,150000000,3.234944,2.695,3,11,True


In [7]:
df.dtypes

movie_id                      int64
movie_popularity            float64
vote_average                float64
vote_count                    int64
budget                        int64
actor_popularity_mean       float64
director_popularity_mean    float64
ratings_enc                   int64
release_month                 int64
profitability                  bool
dtype: object

In [8]:
# Create our features
X = pd.get_dummies(df.drop(columns='profitability'))


# Create our target
y = pd.get_dummies(df['profitability'])

In [9]:
X.describe()

,movie_id,movie_popularity,vote_average,vote_count,budget,actor_popularity_mean,director_popularity_mean,ratings_enc,release_month
count,72.000000,72.000000,72.000000,72.000000,7.200000e+01,72.000000,72.000000,72.000000,72.000000
mean,373058.625000,641.271958,7.427778,6849.861111,1.147528e+08,3.763272,3.496674,3.027778,6.972222
std,231905.622188,1006.994160,0.629604,6819.869858,8.483708e+07,1.819889,2.548206,0.768614,2.973935
min,557.000000,177.734000,6.000000,71.000000,1.000000e+06,1.318528,0.600000,2.000000,1.000000
25%,250246.000000,233.733750,6.975000,1565.500000,3.975000e+07,2.680786,1.765250,3.000000,4.750000
50%,437800.000000,305.311500,7.600000,3982.500000,1.050000e+08,3.208295,2.818500,3.000000,7.000000
75%,554498.000000,624.764500,7.900000,10683.750000,1.850000e+08,4.285281,4.248250,3.000000,9.000000
max,791373.000000,5884.885000,8.400000,25738.000000,3.560000e+08,11.495923,13.329000,6.000000,12.000000


In [10]:
# Check the balance of our target values
y = df['profitability']
y.value_counts()

True     50
False    22
Name: profitability, dtype: int64

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
Counter(y_train)

Counter({False: 15, True: 39})

# Naive Random Oversampling

In [12]:
# Resample the training data with the RandomOversampler
from sklearn.datasets import make_classification
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({False: 39, True: 39})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [14]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[7, 0],
       [4, 7]], dtype=int64)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.8181818181818181

In [16]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.64      1.00      0.64      0.78      0.80      0.66         7
       True       1.00      0.64      1.00      0.78      0.80      0.61        11

avg / total       0.86      0.78      0.86      0.78      0.80      0.63        18



# Undersampling

In [17]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({False: 15, True: 15})

In [18]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [19]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.8181818181818181

In [20]:
# Display the confusion matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[7, 0],
       [5, 6]], dtype=int64)

In [21]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.58      1.00      0.55      0.74      0.74      0.57         7
       True       1.00      0.55      1.00      0.71      0.74      0.52        11

avg / total       0.84      0.72      0.82      0.72      0.74      0.54        18

